In [105]:
import pickle
import numpy as np
import tensorflow

In [106]:
from keras.models import Sequential
from keras.layers import Dense

In [107]:
with open("train_qa.txt","rb") as fp:
    train_data=pickle.load(fp)

In [108]:
with open('test_qa.txt',"rb") as fp:
    test_data=pickle.load(fp)

In [109]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [110]:
type(test_data)

list

In [111]:
' '.join(test_data[0][0])

'Mary got the milk there . John moved to the bedroom .'

In [112]:
' '.join(test_data[0][1])

'Is John in the kitchen ?'

In [113]:
' '.join(test_data[1][0])

'Mary got the milk there . John moved to the bedroom . Mary discarded the milk . John went to the garden .'

In [114]:
' '.join(test_data[1][1])

'Is John in the kitchen ?'

In [115]:
' '.join(test_data[0][2])

'n o'

In [116]:
' '.join(test_data[1][2])

'n o'

In [117]:
' '.join(test_data[0][0][1])

'g o t'

In [118]:
len(train_data)

10000

In [119]:
vocab=set()

In [120]:
all_data=test_data+train_data

In [121]:
for story,question,answer in all_data:
    vocab=vocab.union(set(story))
    vocab=vocab.union(set(question))
    
    
     
  



In [122]:
vocab.add('no')
vocab.add('yes')

In [123]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [124]:
vocab_len=len(vocab)+1

In [125]:
max_story_len=max([len(data[0]) for data in all_data])

In [126]:
max_story_len

156

In [127]:
max_question_len=max([len(data[1]) for data in all_data])

In [128]:
max_question_len

6

In [129]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [130]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()

In [131]:
vocab_size=len(tokenizer.word_index)+1
print("Vocab size:",vocab_size)

Vocab size: 1


In [132]:
import keras

In [133]:
!pip install keras --user

In [134]:
from keras.preprocessing.sequence import pad_sequences

In [135]:
text=['Mary moved to the bathroom,Sandra journeyed to the bedroom.']

In [136]:
tokenizer=Tokenizer(num_words=18)

In [137]:
tokenizer.fit_on_texts(text)

In [138]:
sequences=tokenizer.texts_to_sequences(text)

In [139]:
tokenizer=Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [140]:
tokenizer.word_index

{'mary': 1,
 'garden': 2,
 'bathroom': 3,
 'bedroom': 4,
 'went': 5,
 'left': 6,
 'the': 7,
 'no': 8,
 'john': 9,
 'in': 10,
 '.': 11,
 'is': 12,
 'daniel': 13,
 'got': 14,
 'journeyed': 15,
 'to': 16,
 'picked': 17,
 'discarded': 18,
 'there': 19,
 'moved': 20,
 '?': 21,
 'sandra': 22,
 'kitchen': 23,
 'put': 24,
 'apple': 25,
 'down': 26,
 'football': 27,
 'dropped': 28,
 'yes': 29,
 'office': 30,
 'travelled': 31,
 'took': 32,
 'milk': 33,
 'hallway': 34,
 'back': 35,
 'grabbed': 36,
 'up': 37}

In [141]:
train_story_text=[]
train_question_text=[]
train_answers=[]

In [142]:
for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)

In [143]:
train_story_seq=tokenizer.texts_to_sequences(train_story_text)


In [144]:
len(train_story_text)

10000

In [145]:
len(train_story_seq)

10000

In [146]:
def vectorize_stories(data,word_index=tokenizer.word_index,max_story_len=max_story_len,max_question_len=max_question_len):
    X=[]
    Xq=[]
    Y=[]
    for story,query,answer in data:
        if'<unk>' not in word_index:
            word_index['<unk>']=len(word_index)+1
        x=[word_index.get(word.lower(),word_index['<unk>']) for word in story]
        xq=[word_index.get(word.lower(),word_index['<unk>']) for word in query]
        y=np.zeros(len(word_index)+1)
        y[word_index[answer]]=1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X,maxlen=max_story_len),pad_sequences(Xq,maxlen=max_question_len),np.array(Y))

In [147]:
inputs_train,queries_train,answers_train=vectorize_stories(train_data)

In [148]:
inputs_train.shape

(10000, 156)

In [149]:
answers_train.shape

(10000, 39)

In [150]:
inputs_test,queries_test,answers_test=vectorize_stories(test_data)

In [151]:
inputs_test

array([[ 0,  0,  0, ...,  7,  4, 11],
       [ 0,  0,  0, ...,  7,  2, 11],
       [ 0,  0,  0, ...,  7,  2, 11],
       ...,
       [ 0,  0,  0, ...,  7, 25, 11],
       [ 0,  0,  0, ...,  7,  2, 11],
       [ 0,  0,  0, ..., 25, 19, 11]], shape=(1000, 156), dtype=int32)

In [152]:
queries_test

array([[12,  9, 10,  7, 23, 21],
       [12,  9, 10,  7, 23, 21],
       [12,  9, 10,  7,  2, 21],
       ...,
       [12,  1, 10,  7,  4, 21],
       [12, 22, 10,  7,  2, 21],
       [12,  1, 10,  7,  2, 21]], shape=(1000, 6), dtype=int32)

In [153]:
answers_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(1000, 39))

In [154]:
sum(answers_test)

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 503.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0., 497.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.])

In [155]:
tokenizer.word_index['yes']

29

In [156]:
tokenizer.word_index['no']

8

In [157]:
from keras.models import Sequential, Model
from keras.layers import Embedding
from keras.layers import Input,Activation,Dense,Permute,Dropout
from keras.layers import add,dot,concatenate
from keras.layers import LSTM 

In [158]:
input_sequence=Input((max_story_len,))
question=Input((max_question_len,))

In [159]:
input_encoder_m=Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

In [160]:
input_encoder_c=Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))

In [161]:
question_encoder=Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,output_dim=64,input_length=max_question_len))
question_encoder.add(Dropout(0.3))

In [162]:
input_encoded_m=input_encoder_m(input_sequence)
input_encoded_c=input_encoder_c(input_sequence)
question_encoded=question_encoder(question)

In [163]:
match=dot([input_encoded_m,question_encoded],axes=(2,2))
match=Activation('softmax')(match)

In [164]:
response=add([match,input_encoded_c])
response=Permute((2,1))(response)

In [165]:
answer=concatenate([response,question_encoded])

In [166]:
answer=LSTM(32,return_sequences=False)(answer)

In [167]:
answer

<KerasTensor shape=(None, 32), dtype=float32, sparse=False, ragged=False, name=keras_tensor_84>

In [168]:
answer=Dropout(0.5)(answer)
answer=Dense(vocab_size)(answer)

In [172]:
answer=Activation('softmax')(answer)
model=Model([input_sequence,question],answer)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [173]:
from tensorflow.keras.layers import Input,Dense,LSTM,Embedding,concatenate
from tensorflow.keras.models import Model
max_len_input=inputs_train.shape[1]
max_len_query=queries_train.shape[1]
input1=Input(shape=(max_len_input,))
x1=Embedding(vocab_size,64)(input1)
x1=LSTM(32)(x1)
input2=Input(shape=(max_len_query,))
x2=Embedding(vocab_size,64)(input2)
x2=LSTM(32)(x2)
merged=concatenate([x1,x2])
output=Dense(40,activation='softmax')(merged)
model=Model(inputs=[input1,input2],outputs=output)
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_26 (InputLayer)   │ (None, 156)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_27 (InputLayer)   │ (None, 6)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_19 (Embedding)      │ (None, 156, 64)           │              64 │ input_layer_26[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_20 (Embedding)      │ (None, 6, 64)             │              64 │ input_layer_27[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_14 (LSTM)                │ (None, 32)                │          12,416 │ embedding_19[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_15 (LSTM)                │ (None, 32)                │          12,416 │ embedding_20[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_7 (Concatenate)   │ (None, 64)                │               0 │ lstm_14[0][0],             │
│                               │                           │                 │ lstm_15[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_7 (Dense)               │ (None, 40)                │           2,600 │ concatenate_7[0][0]        │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 27,560 (107.66 KB)

 Trainable params: 27,560 (107.66 KB)

 Non-trainable params: 0 (0.00 B)

In [174]:
history=model.fit([inputs_train,queries_train],answers_train,batch_size=32,epochs=30,validation_data=([inputs_test,queries_test],answers_test))

Epoch 1/30


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 39), output.shape=(None, 40)

In [175]:
print("vocab_size =", vocab_size)

vocab_size = 1


In [176]:
vocab_size = int(
    max(inputs_train.max(), queries_train.max())
) + 1

print("FIXED vocab_size =", vocab_size)

FIXED vocab_size = 38


In [177]:
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, concatenate
from tensorflow.keras.models import Model

max_len_input = inputs_train.shape[1]
max_len_query = queries_train.shape[1]

input1 = Input(shape=(max_len_input,))
x1 = Embedding(input_dim=vocab_size, output_dim=64)(input1)
x1 = LSTM(32)(x1)

input2 = Input(shape=(max_len_query,))
x2 = Embedding(input_dim=vocab_size, output_dim=64)(input2)
x2 = LSTM(32)(x2)

merged = concatenate([x1, x2])
output = Dense(40, activation='softmax')(merged)

model = Model(inputs=[input1, input2], outputs=output)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_28 (InputLayer)   │ (None, 156)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_29 (InputLayer)   │ (None, 6)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_21 (Embedding)      │ (None, 156, 64)           │           2,432 │ input_layer_28[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_22 (Embedding)      │ (None, 6, 64)             │           2,432 │ input_layer_29[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_16 (LSTM)                │ (None, 32)                │          12,416 │ embedding_21[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_17 (LSTM)                │ (None, 32)                │          12,416 │ embedding_22[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_8 (Concatenate)   │ (None, 64)                │               0 │ lstm_16[0][0],             │
│                               │                           │                 │ lstm_17[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_8 (Dense)               │ (None, 40)                │           2,600 │ concatenate_8[0][0]        │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 32,296 (126.16 KB)

 Trainable params: 32,296 (126.16 KB)

 Non-trainable params: 0 (0.00 B)

In [178]:
history = model.fit(
    [inputs_train, queries_train],
    answers_train,
    batch_size=32,
    epochs=30,
    validation_data=([inputs_test, queries_test], answers_test)
)

Epoch 1/30


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 39), output.shape=(None, 40)

In [179]:
num_classes = answers_train.shape[1]
print("Number of classes in labels =", num_classes)

Number of classes in labels = 39


In [180]:
output = Dense(num_classes, activation='softmax')(merged)

In [181]:
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, concatenate
from tensorflow.keras.models import Model

vocab_size = int(max(inputs_train.max(), queries_train.max())) + 1

max_len_input = inputs_train.shape[1]
max_len_query = queries_train.shape[1]

input1 = Input(shape=(max_len_input,))
x1 = Embedding(input_dim=vocab_size, output_dim=64)(input1)
x1 = LSTM(32)(x1)

input2 = Input(shape=(max_len_query,))
x2 = Embedding(input_dim=vocab_size, output_dim=64)(input2)
x2 = LSTM(32)(x2)

merged = concatenate([x1, x2])

num_classes = answers_train.shape[1]
output = Dense(num_classes, activation='softmax')(merged)

model = Model(inputs=[input1, input2], outputs=output)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_30 (InputLayer)   │ (None, 156)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_31 (InputLayer)   │ (None, 6)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_23 (Embedding)      │ (None, 156, 64)           │           2,432 │ input_layer_30[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_24 (Embedding)      │ (None, 6, 64)             │           2,432 │ input_layer_31[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_18 (LSTM)                │ (None, 32)                │          12,416 │ embedding_23[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_19 (LSTM)                │ (None, 32)                │          12,416 │ embedding_24[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_9 (Concatenate)   │ (None, 64)                │               0 │ lstm_18[0][0],             │
│                               │                           │                 │ lstm_19[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_10 (Dense)              │ (None, 39)                │           2,535 │ concatenate_9[0][0]        │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 32,231 (125.90 KB)

 Trainable params: 32,231 (125.90 KB)

 Non-trainable params: 0 (0.00 B)

In [182]:
history = model.fit(
    [inputs_train, queries_train],
    answers_train,
    batch_size=32,
    epochs=30,
    validation_data=([inputs_test, queries_test], answers_test)
)

Epoch 1/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 28s 65ms/step - accuracy: 0.5009 - loss: 0.8723 - val_accuracy: 0.4970 - val_loss: 0.6946
Epoch 2/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 65ms/step - accuracy: 0.5065 - loss: 0.6956 - val_accuracy: 0.4970 - val_loss: 0.6952
Epoch 3/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 63ms/step - accuracy: 0.5004 - loss: 0.6971 - val_accuracy: 0.5030 - val_loss: 0.6947
Epoch 4/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 62ms/step - accuracy: 0.4949 - loss: 0.6968 - val_accuracy: 0.5030 - val_loss: 0.6984
Epoch 5/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 65ms/step - accuracy: 0.5045 - loss: 0.6951 - val_accuracy: 0.4970 - val_loss: 0.6938
Epoch 6/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 65ms/step - accuracy: 0.5045 - loss: 0.6965 - val_accuracy: 0.5030 - val_loss: 0.6959
Epoch 7/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 64ms/step - accuracy: 0.5025 - loss: 0.6961 - val_accuracy: 0.5060 - val_loss: 0.6931
Epoch 8/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 65ms/step - accuracy: 0.4931 - loss: 0.6969 - 

In [193]:
filename='chatbot_120_epochs.keras'
model.save(filename)

In [194]:
model.load_weights(filename)
pred_results=model.predict(([inputs_test,queries_test]))

32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step


In [195]:
story=' '.join(word for word in test_data[0][0])
print(story)

Mary got the milk there . John moved to the bedroom .


In [196]:
query=' '.join(word for word in test_data[0][1])
print(query)

Is John in the kitchen ?


In [197]:
print("True test Answer from data is:",test_data[0][2])

True test Answer from data is: no


In [216]:
val_max=np.argmax(pred_results[0])
for key,val in tokenizer.word_index.items():
    if val==val_max:
        k=key
print("predicted answer is:",k)
print("probability of certainity was: ",pred_results[0][val_max])
    

predicted answer is: no
probability of certainity was:  0.510423


In [211]:
my_story="john travelled the bathroom last saturday. Sandra dropped the football in the garden."
my_story

'john travelled the bathroom last saturday. Sandra dropped the football in the garden.'

In [212]:
my_question="did john travelled the bathroom last friday?"

In [213]:
my_question.split()

['did', 'john', 'travelled', 'the', 'bathroom', 'last', 'friday?']

In [214]:
my_data=[(my_story,my_question,'yes')]

In [215]:
my_story,my_ques,my_ans=vectorize_stories(my_data)

In [205]:
pred_results=model.predict(([my_story,my_ques]))

InvalidArgumentError: Graph execution error:

Detected at node functional_16_1/embedding_24_1/GatherV2 defined at (most recent call last):
  File "C:\Users\Administrator\anaconda3\Lib\runpy.py", line 198, in _run_module_as_main

  File "C:\Users\Administrator\anaconda3\Lib\runpy.py", line 88, in _run_code

  File "C:\Users\Administrator\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\Administrator\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\Administrator\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\Administrator\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "C:\Users\Administrator\anaconda3\Lib\asyncio\base_events.py", line 683, in run_forever

  File "C:\Users\Administrator\anaconda3\Lib\asyncio\base_events.py", line 2050, in _run_once

  File "C:\Users\Administrator\anaconda3\Lib\asyncio\events

In [220]:
print("max my_story:", my_story.max())
print("max my_ques:", my_ques.max())
print("vocab_size:", vocab_size)

max my_story: 38
max my_ques: 38
vocab_size: 38


In [221]:
import numpy as np

my_story = np.where(my_story >= vocab_size, 0, my_story)
my_ques  = np.where(my_ques  >= vocab_size, 0, my_ques)

In [222]:
my_story = my_story.reshape(1, -1)
my_ques  = my_ques.reshape(1, -1)

In [223]:
pred_results = model.predict([my_story, my_ques])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
